# Import necessary libraries

In [1]:
import os

In [2]:
# move working dir up to parent, allows us to import from utils without too many shenanigans
os.chdir(os.pardir)

In [3]:
import copy
import datetime
import pickle
from math import pow, sqrt

import networkx as nx
import numpy as np
import pandas as pd
from matplotlib import colormaps as cmap
from matplotlib.pyplot import figure
from tqdm import tqdm

from sklearn.cluster import (SpectralClustering, DBSCAN, AgglomerativeClustering, OPTICS)

from utils.data_generated import (load_is_in_movies, load_movie_metadata, load_people)


# Functions used for generating, clustering, displaying, saving, and loading graphs

In [4]:
# This code block contains the functions necessary to generate a graph

# Helper function that combines all the movies a person has worked on
def combine_movies(id, period, isinmovies):
    distinct_movies = set()
    entry = isinmovies[isinmovies.index == id]
    entry = entry[entry.columns[~entry.isnull().all()]]
    for job in entry:
        movies_set = set()
        movies = entry[job].values[0]
        for movie in movies:
            if movie in period:
                movies_set.add(movie)
        if len(movies_set) > 0:
            distinct_movies = distinct_movies.union(movies_set)
    return distinct_movies

# Creates an empty graph
def initialize_graph():
    print('Initializing Graph!')
    return nx.Graph()

# Generates nodes for the graph and assigns them attributes
def compute_nodes(G, moviesinperiod, people, isinmovies):
    print('Processing People into Nodes!')
    period = moviesinperiod['title_id_imdb'].values

    with tqdm(total=len(people)) as progress_bar:
        for _, row in tqdm(people.iterrows()):
            id = row['person_name_id']
            name = row['person_name']
            movies = combine_movies(id, period, isinmovies)
                
            if len(movies) == 0:
                pass
            elif len(movies) >= NODE_THRESHOLD:
                G.add_node(id, size=NODE_BASE_SIZE*len(movies), num_connections=0, alpha=NODE_BASE_ALPHA, name=name, movies=movies, color='red')
            else:
                G.add_node(id, size=NODE_BASE_SIZE*len(movies), num_connections=0, alpha=NODE_BASE_ALPHA, name=name, movies=movies, color='blue')
            progress_bar.update(1)

# For every pair of nodes computes the edge thickness and updates the alpha of the node color
def compute_edges(G):
    print("Computing Edges!")

    with tqdm(total=G.order()) as progress_bar:
        for node1 in tqdm(G.nodes(data='movies')):
            id1 = node1[0]
            movies1 = node1[1]

            for node2 in G.nodes(data='movies'):
                id2 = node2[0]            

                if not G.has_edge(id1, id2) and id1 != id2:
                    movies2 = node2[1]
                    common_movies = movies1.intersection(movies2)
                    
                    if len(common_movies) > 0:

                        G.add_edge(id1, id2, weight=len(common_movies) * EDGE_BASE_WEIGHT, color='lightgray')
                        G.nodes[id1]['num_connections'] = G.nodes[id1]['num_connections'] + len(common_movies)
                        G.nodes[id2]['num_connections'] = G.nodes[id2]['num_connections'] + len(common_movies)
            progress_bar.update(1)

#Performs all steps of graph generation
def generate_graph(moviesinperiod, people, isinmovies):
    G = initialize_graph()
    compute_nodes(G, moviesinperiod, people, isinmovies)
    compute_edges(G)
    print(f'Resulting Graph has {G.order()} Nodes and {G.size()} Edges!')
    return G


In [5]:
# This code block contains the functions necessary to cluster a graph

# Generates a copy of the original graph with updated colors to indicate cluster coefficients
def generate_cluster_coefficient_graph(G, clustered_nodes, colormap):
    G_clustered = copy.deepcopy(G)
    for node in G_clustered.nodes(data=True):
        if clustered_nodes[node[0]] >= 0:
            node[1]['color'] = cmap[colormap](clustered_nodes[node[0]])
    return G_clustered

# Generates a copy of the original graph with updated colors to indicate clusters
def generate_clustered_graph(G, clustered_nodes, colormap):
    G_clustered = copy.deepcopy(G)
    clusters = np.unique(list(clustered_nodes.values()))
    cluster_colors = np.linspace(start=0, stop=1, num=max(clustered_nodes.values())+1)
    np.random.default_rng().shuffle(cluster_colors)
    nodes_in_clusters = {}
    print(f'Number of clusters is {max(clustered_nodes.values())+1}!')
    
    # init empty list for each cluster
    for cluster in clusters:
        nodes_in_clusters[cluster] = []

    # assign nodes to clusters
    print('Grouping nodes into respective clusters!')
    for node, cluster in clustered_nodes.items():
        nodes_in_clusters[cluster].append(node)

    # recolor every cluster accordingly
    print('Recoloring nodes and edges according to the clustering!')
    color_index = 0
    for cluster in clusters:
        sub_G = G_clustered.subgraph(nodes_in_clusters[cluster])
        # cluster_color = cluster / max(clustered_nodes.values())
        cluster_color = cluster_colors[color_index]
        if cluster >= 0:
            print(f'Cluster number {cluster} has {sub_G.order()} nodes and {sub_G.size()} edges!')
            for node in sub_G.nodes(data=True):
                node[1]['color'] = cmap[colormap](cluster_color)
            for edge in sub_G.edges(data=True):
                edge[2]['color'] = cmap[colormap](cluster_color)
            color_index += 1
        else:
            print(f'There are {sub_G.order()} outliers!')
            for node in sub_G.nodes(data=True):
                node[1]['color'] = NODE_OUTLIER_COLOR

    return G_clustered

# Computes the clustering coefficient for nodes
def compute_clustering_coefficient(G):
    return nx.clustering(G)

# Clusters node using spectral clustering
def spectral_clustering(node_positions, n_clusters, n_iter, seed):
    positions_array = np.array([position for position in node_positions.values()])
    clustering = SpectralClustering(n_clusters=n_clusters,
        random_state=seed, n_init=n_iter, assign_labels='kmeans'
        ).fit(positions_array)
    labels = clustering.labels_
    
    return {id:label for id, label in zip(node_positions.keys(), labels)}

# Clusters node using dbscan
def dbscan_clustering(node_positions, eps, min_samples):
    positions_array = np.array([position for position in node_positions.values()])
    clustering = DBSCAN(eps=eps,
        min_samples=min_samples, metric='euclidean', n_jobs=1
        ).fit(positions_array)
    labels = clustering.labels_

    return {id:label for id, label in zip(node_positions.keys(), labels)}

# Clusters node using hierarchial clustering
def hierarchial_clustering(node_positions, n_clusters, distance_threshold):
    positions_array = np.array([position for position in node_positions.values()])
    clustering = AgglomerativeClustering(n_clusters=n_clusters,
        linkage='ward', distance_threshold=distance_threshold
        ).fit(positions_array)
    labels = clustering.labels_
    
    return {id:label for id, label in zip(node_positions.keys(), labels)}

# Clusters node using optics
def optics_clustering(node_positions, min_samples, max_eps, xi, min_cluster_size):
    positions_array = np.array([position for position in node_positions.values()])
    clustering = OPTICS(min_samples=min_samples, max_eps=max_eps,
        xi=xi, min_cluster_size=min_cluster_size, n_jobs=10
        ).fit(positions_array)
    labels = clustering.labels_
    
    return {id:label for id, label in zip(node_positions.keys(), labels)}

In [6]:
# This code block contains the functions necessary to display a graph

# Position nodes using Fruchterman-Reingold force-directed algorithm
def compute_node_positions(G, n_iter, seed):
    print('Computing Node Layout! (Warning: may take a long time)')
    return nx.spring_layout(G, k=SPRING_COEFFICIENT/sqrt(G.order()),
        iterations=n_iter, weight='weight', dim=2, seed=seed)

# Returns nodes for which the labels will be displayed
def compute_heavy_nodes(G):
    heavy_nodes = []
    for node in G.nodes(data=True):
        if node[1]['size'] >= LABEL_THRESHOLD:
            heavy_nodes.append(node)
    return heavy_nodes

# Computes all the parameters needed to draw the graph apart from the node positions and the graph itself
def compute_params(G):
    nodes = G.nodes()
    node_sizes = [sqrt(x) for x in list(nx.get_node_attributes(G, 'size').values())]
    node_colors = list(nx.get_node_attributes(G, 'color').values())
    # node_alphas = list(nx.get_node_attributes(G, 'alpha').values())
    node_alphas = [pow(NODE_BASE_ALPHA, pow(NODE_ALPHA_EXP_COEF, x)) for x in list(nx.get_node_attributes(G, 'num_connections').values())]

    edges = G.edges()
    edge_weights = list(nx.get_edge_attributes(G, 'weight').values())
    edge_colors = list(nx.get_edge_attributes(G, 'color').values())
    edge_alphas = EDGE_ALPHA

    heavy_nodes = compute_heavy_nodes(G)
    subgraph = G.subgraph([node[0] for node in heavy_nodes])
    labels = {node[0]:node[1]['name'] for node in heavy_nodes}
    font_sizes = LABEL_SIZE
    font_colors = LABEL_COLOR
    font_alphas = LABEL_ALPHA
    vertical_alignment = LABEL_ALIGNMENT

    return {'node_params':{'ids':nodes, 'sizes':node_sizes, 'colors':node_colors, 'alphas':node_alphas},
            'edge_params':{'ids':edges, 'weights':edge_weights, 'colors':edge_colors, 'alphas':edge_alphas},
            'label_params':{'subgraph':subgraph, 'labels':labels, 'sizes':font_sizes, 'colors':font_colors, 'alphas':font_alphas, 'vertical_alignments':vertical_alignment}}

# Initializes an empty figure and returns the handle
def init_graph(size):
    return figure(figsize=size)

# Draws nodes according to params
def draw_nodes(G, node_positions, params):
    nx.draw_networkx_nodes(G, node_positions,
                        nodelist=params['ids'],
                        node_size=params['sizes'],
                        node_color=params['colors'],
                        alpha=params['alphas'])

# Draws edges according to params
def draw_edges(G, node_positions, params):
    nx.draw_networkx_edges(G, node_positions,
                        edgelist=params['ids'],
                        width=params['weights'],
                        edge_color=params['colors'],
                        alpha=params['alphas'])

# Draws labels according to params
def draw_labels(node_positions, params):
    nx.draw_networkx_labels(params['subgraph'], node_positions,
                            labels=params['labels'],
                            font_size=params['sizes'],
                            font_color=params['colors'],
                            alpha=params['alphas'],
                            verticalalignment=params['vertical_alignments'])

# Draws the graph on a figure and returns the handle to the figure
def draw_graph(G, node_positions, size, params):
    fig = init_graph(size)
    draw_nodes(G, node_positions, params['node_params'])
    draw_edges(G, node_positions, params['edge_params'])
    draw_labels(node_positions, params['label_params'])

    return fig

In [7]:
# This code block contains the functions necessary to pickle/unpickle graphs and related objects
# that take a long time to compute. Also provides a function for saving figures.

# File directories
PATH_GRAPHS = '../generated/graphs/'
PATH_FIGURES = '../results/graphs/'

# Filename suffixes
SUFFIX_GRAPHS = '_graph.pickle'
SUFFIX_POSITIONS = '_pos.pickle'
SUFFIX_CLUSTER_COEFFICIENT = '_cluster_coefficient.pickle'
SUFFIX_CLUSTER_SPECTRAL = '_cluster_spectral.pickle'
SUFFIX_CLUSTER_DBSCAN = '_cluster_dbscan.pickle'
SUFFIX_CLUSTER_HIERARCHIAL = '_cluster_hierarchial.pickle'
SUFFIX_CLUSTER_OPTICS = '_cluster_optics.pickle'

SUFFIX_FIGURES = '.png'
SUFFIX_CLUSTER_COEFFICIENT_FIGURE = '_cluster_coefficient.png'
SUFFIX_CLUSTER_SPECTRAL_FIGURE = '_cluster_spectral.png'
SUFFIX_CLUSTER_DBSCAN_FIGURE = '_cluster_dbscan.png'
SUFFIX_CLUSTER_HIERARCHIAL_FIGURE = '_cluster_hierarchial.png'
SUFFIX_CLUSTER_OPTICS_FIGURE = '_cluster_optics.png'

# Pickle objects
def pickle_graph(G, filename):
    with open(PATH_GRAPHS + filename + SUFFIX_GRAPHS, 'wb') as handle:
        pickle.dump(G, handle, protocol=pickle.HIGHEST_PROTOCOL)

def pickle_positions(pos, filename):
    with open(PATH_GRAPHS + filename + SUFFIX_POSITIONS, 'wb') as handle:
        pickle.dump(pos, handle, protocol=pickle.HIGHEST_PROTOCOL)

def pickle_cluster_coefficients(clustered_nodes, filename):
    with open(PATH_GRAPHS + filename + SUFFIX_CLUSTER_COEFFICIENT, 'wb') as handle:
        pickle.dump(clustered_nodes, handle, protocol=pickle.HIGHEST_PROTOCOL)

def pickle_cluster_spectral(clustered_nodes, filename):
    with open(PATH_GRAPHS + filename + SUFFIX_CLUSTER_SPECTRAL, 'wb') as handle:
        pickle.dump(clustered_nodes, handle, protocol=pickle.HIGHEST_PROTOCOL)

def pickle_cluster_dbscan(clustered_nodes, filename):
    with open(PATH_GRAPHS + filename + SUFFIX_CLUSTER_DBSCAN, 'wb') as handle:
        pickle.dump(clustered_nodes, handle, protocol=pickle.HIGHEST_PROTOCOL)

def pickle_cluster_hierarchial(clustered_nodes, filename):
    with open(PATH_GRAPHS + filename + SUFFIX_CLUSTER_HIERARCHIAL, 'wb') as handle:
        pickle.dump(clustered_nodes, handle, protocol=pickle.HIGHEST_PROTOCOL)

def pickle_cluster_optics(clustered_nodes, filename):
    with open(PATH_GRAPHS + filename + SUFFIX_CLUSTER_OPTICS, 'wb') as handle:
        pickle.dump(clustered_nodes, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Unpickle objects
def unpickle_graph(filename):
    with open(PATH_GRAPHS + filename + SUFFIX_GRAPHS, 'rb') as handle:
        return pickle.load(handle)

def unpickle_positions(filename):
    with open(PATH_GRAPHS + filename + SUFFIX_POSITIONS, 'rb') as handle:
        return pickle.load(handle)

def unpickle_cluster_coefficients(filename):
    with open(PATH_GRAPHS + filename + SUFFIX_CLUSTER_COEFFICIENT, 'rb') as handle:
        return pickle.load(handle)

def unpickle_cluster_spectral(filename):
    with open(PATH_GRAPHS + filename + SUFFIX_CLUSTER_SPECTRAL, 'rb') as handle:
        return pickle.load(handle)

def unpickle_cluster_dbscan(filename):
    with open(PATH_GRAPHS + filename + SUFFIX_CLUSTER_DBSCAN, 'rb') as handle:
        return pickle.load(handle)

def unpickle_cluster_hierarchial(filename):
    with open(PATH_GRAPHS + filename + SUFFIX_CLUSTER_HIERARCHIAL, 'rb') as handle:
        return pickle.load(handle)

def unpickle_cluster_optics(filename):
    with open(PATH_GRAPHS + filename + SUFFIX_CLUSTER_OPTICS, 'rb') as handle:
        return pickle.load(handle)

# Save figures
def save_figure(fig, filename):
    fig.savefig(PATH_FIGURES + filename + SUFFIX_FIGURES)

def save_cluster_coefficient_figure(fig, filename):
    fig.savefig(PATH_FIGURES + filename + SUFFIX_CLUSTER_COEFFICIENT_FIGURE)

def save_cluster_spectral_figure(fig, filename):
    fig.savefig(PATH_FIGURES + filename + SUFFIX_CLUSTER_SPECTRAL_FIGURE)

def save_cluster_dbscan_figure(fig, filename):
    fig.savefig(PATH_FIGURES + filename + SUFFIX_CLUSTER_DBSCAN_FIGURE)

def save_cluster_hierarchial_figure(fig, filename):
    fig.savefig(PATH_FIGURES + filename + SUFFIX_CLUSTER_HIERARCHIAL_FIGURE)

def save_cluster_optics_figure(fig, filename):
    fig.savefig(PATH_FIGURES + filename + SUFFIX_CLUSTER_OPTICS_FIGURE)

# Parameter definitions and loading the datasets

In [8]:
# Decide the start date (inclusive) and start date (exclusive)
# that will be used to prune the dataframe
START_DATE = pd.to_datetime(datetime.date(1996,1,1))
END_DATE = pd.to_datetime(datetime.date(2012,1,1))

In [9]:
# Import the necessary dataframes for graph generation
# and prune the movies dataframe to include only the given time frame
movie_df = load_movie_metadata()
movie_df = movie_df.loc[(movie_df['release_date'] >= START_DATE) & (movie_df['release_date'] < END_DATE)]
people_df = load_people()
isinmovies_df = load_is_in_movies()

In [8]:
# Following are the constants used in graph generation/display

# node params
NODE_BASE_SIZE = 1          # Base size of a node (increases if a person is more active)
NODE_BASE_ALPHA = 0.01      # Base alpha of a node color (increases if a person has many connections)
NODE_ALPHA_EXP_COEF = 0.97  # Parameter that controls how fast node alpha increases for each connection (Smaller param => Faster convergence to 1)
NODE_THRESHOLD = 20         # Parameter that controls how active a person needs to be for their node color to change
NODE_OUTLIER_COLOR = 'black'# Color assigned to outlier nodes

# edge params
EDGE_BASE_WEIGHT = 0.1      # Base edge thickness (increases if two people work together often)
EDGE_ALPHA = 0.03           # Edge color alpha

# label params
LABEL_SIZE = 8              # Font size of the labels
LABEL_COLOR = 'black'       # Font color of the labels
LABEL_ALPHA = 1             # Font alpha of the labels
LABEL_ALIGNMENT = 'bottom'  # Font alignment of the labels
LABEL_THRESHOLD = 20        # Parameter that controls how active a person needs to be for their name to be displayed

# general params
SPRING_COEFFICIENT = 1      # Indicates how much the nodes should be forced to spread apart on the graph (default=1)
COLOR_MAP = 'plasma'        # Indicates which matplotlib colormap to use for cluster coloring
SEED = 1                    # Fix seed so that the results are reproducible
FIG_SIZE = (320,320)        # Size of the figure to be drawn

# Generating, saving and displaying the graph

In [11]:
# Common filename for graphs, positions and figures to be saved as
filename = '19960101-20120101'

In [12]:
# Generate the graph
G = generate_graph(movie_df, people_df, isinmovies_df)
pickle_graph(G, filename)

Initializing Graph!
Processing People into Nodes!


74973it [12:15, 101.95it/s]3 [12:15<00:00, 116.70it/s]
100%|██████████| 74973/74973 [12:15<00:00, 101.95it/s]


Computing Edges!


100%|██████████| 71740/71740 [1:36:29<00:00, 12.39it/s]


Resulting Graph has 71740 Nodes and 562277 Edges!


In [13]:
# Compute node positions using Fruchterman-Reingold force-directed algorithm
node_positions = compute_node_positions(G=G, n_iter=50, seed=SEED)
pickle_positions(node_positions, filename)

Computing Node Layout! (Warning: may take a long time)


In [ ]:
# Display the graph
params = compute_params(G)
fig = draw_graph(G, node_positions, FIG_SIZE, params)
save_figure(fig, filename)

# Clustering Coefficient

In [11]:
# Compute clustering coefficient graph
clustered_nodes = compute_clustering_coefficient(G)
G_clustered = generate_cluster_coefficient_graph(G, clustered_nodes, COLOR_MAP)
pickle_cluster_coefficients(clustered_nodes, filename)

In [ ]:
# Display clustering coefficient graph
params = compute_params(G_clustered)
fig = draw_graph(G_clustered, node_positions, FIG_SIZE, params)
save_cluster_coefficient_figure(fig, filename)

# Spectral Clustering

Infeasable for a graph of our size

In [13]:
# # Compute spectral clustering graph
# clustered_nodes = spectral_clustering(node_positions=node_positions, n_clusters=4, n_iter=10, seed=SEED)
# G_clustered = generate_clustered_graph(G, clustered_nodes, COLOR_MAP)
# pickle_cluster_spectral(clustered_nodes, filename)

In [ ]:
# # Display spectral clustering graph
# params = compute_params(G_clustered)
# fig = draw_graph(G_clustered, node_positions, FIG_SIZE, params)
# save_cluster_spectral_figure(fig, filename)

# DBSCAN

In [13]:
# Compute dbscan clustering graph
clustered_nodes = dbscan_clustering(node_positions=node_positions, eps=0.0029, min_samples=15)
G_clustered = generate_clustered_graph(G, clustered_nodes, COLOR_MAP)
pickle_cluster_dbscan(clustered_nodes, filename)

Number of clusters is 388!
Grouping nodes into respective clusters!
Recoloring nodes and edges according to the clustering!
There are 26639 outliers!
Cluster number 0 has 25417 nodes and 244092 edges!
Cluster number 1 has 56 nodes and 126 edges!
Cluster number 2 has 201 nodes and 325 edges!
Cluster number 3 has 64 nodes and 124 edges!
Cluster number 4 has 138 nodes and 254 edges!
Cluster number 5 has 2324 nodes and 23143 edges!
Cluster number 6 has 38 nodes and 63 edges!
Cluster number 7 has 20 nodes and 46 edges!
Cluster number 8 has 300 nodes and 524 edges!
Cluster number 9 has 298 nodes and 570 edges!
Cluster number 10 has 46 nodes and 62 edges!
Cluster number 11 has 32 nodes and 67 edges!
Cluster number 12 has 107 nodes and 247 edges!
Cluster number 13 has 1219 nodes and 6514 edges!
Cluster number 14 has 26 nodes and 18 edges!
Cluster number 15 has 24 nodes and 53 edges!
Cluster number 16 has 19 nodes and 21 edges!
Cluster number 17 has 51 nodes and 73 edges!
Cluster number 18 has 

In [ ]:
# Display dbscan clustering graph
params = compute_params(G_clustered)
fig = draw_graph(G_clustered, node_positions, FIG_SIZE, params)
save_cluster_dbscan_figure(fig, filename)

# Hierarchial Clustering

Infeasable for a graph of our size

In [ ]:
# # Compute hierarchial clustering graph
# clustered_nodes = hierarchial_clustering(node_positions=node_positions, n_clusters=None, distance_threshold=10)
# G_clustered = generate_clustered_graph(G, clustered_nodes, COLOR_MAP)
# pickle_cluster_hierarchial(clustered_nodes, filename)

In [ ]:
# # Display hierarchial clustering graph
# params = compute_params(G_clustered)
# fig = draw_graph(G_clustered, node_positions, FIG_SIZE, params)
# save_cluster_hierarchial_figure(fig, filename)

# OPTICS

In [15]:
# Compute optics clustering graph
clustered_nodes = optics_clustering(node_positions=node_positions, min_samples=15, max_eps=0.0029, xi=0.05, min_cluster_size=400)
G_clustered = generate_clustered_graph(G, clustered_nodes, COLOR_MAP)
pickle_cluster_optics(clustered_nodes, filename)

Number of clusters is 5!
Grouping nodes into respective clusters!
Recoloring nodes and edges according to the clustering!
There are 38299 outliers!
Cluster number 0 has 25417 nodes and 244092 edges!
Cluster number 1 has 2324 nodes and 23143 edges!
Cluster number 2 has 1219 nodes and 6514 edges!
Cluster number 3 has 1140 nodes and 6881 edges!
Cluster number 4 has 3341 nodes and 39117 edges!


In [ ]:
# Display optics clustering graph
params = compute_params(G_clustered)
fig = draw_graph(G_clustered, node_positions, FIG_SIZE, params)
save_cluster_optics_figure(fig, filename)

# Loading the graph

In [9]:
# Uncomment the following lines to load a saved graph/position/clustering

# filename = '19960101-20120101'
# G = unpickle_graph(filename)
# node_positions = unpickle_positions(filename)
# cluster_coefficients = unpickle_cluster_coefficients(filename)
# cluster_spectral = unpickle_cluster_spectral(filename)
# cluster_dbscan = unpickle_cluster_dbscan(filename)
# cluster_hierarchial = unpickle_cluster_hierarchial(filename)
# cluster_optics = unpickle_cluster_optics(filename)